In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyomo.environ import *
from random import shuffle
np.random.seed(0)

In [2]:
def create_model(data):
    
    # unpacking data
    nrakes = len(data['a'])
    ndest = data['pendency'].shape[0]
    nterm = data['pendency'].shape[1]
    BigM = data['BigM']
    BigM2 = data['BigM2']
    
    M = ConcreteModel()
    
    # Adding variables
    M.Sa = Var(RangeSet(nrakes),RangeSet(len(data['sys'])),domain=NonNegativeReals)
    M.Sb = Var(RangeSet(nrakes),RangeSet(len(data['sys'])),domain=NonNegativeReals) 
    M.xa = Var(RangeSet(int(nrakes*(nrakes-1)/2)),RangeSet(len(data['sys'])),domain=Binary)
    M.xb = Var(RangeSet(int(nrakes*(nrakes-1)/2)),RangeSet(len(data['sys'])),domain=Binary)  
    M.w1a = Var(RangeSet(nrakes),RangeSet(data['sys'][0]),domain=Binary)
    M.w2a = Var(RangeSet(nrakes),RangeSet(data['sys'][1]),domain=Binary)
    M.w4a = Var(RangeSet(nrakes),RangeSet(sum(data['sys'][3])),domain=Binary)
    M.w1b = Var(RangeSet(nrakes),RangeSet(data['sys'][1]),domain=Binary)
    M.w2b = Var(RangeSet(nrakes),RangeSet(data['sys'][1]),domain=Binary)
    M.y = Var(RangeSet(nrakes),RangeSet(nterm),domain=Binary)
    M.v = Var(RangeSet(nrakes),RangeSet(ndest),domain=Binary)
    M.g = Var(RangeSet(nrakes),RangeSet(nterm),domain=NonNegativeIntegers,bounds=(0,data['R'][1]))
    M.n = Var(RangeSet(nrakes),RangeSet(ndest),RangeSet(nterm),domain=NonNegativeIntegers,bounds=(0,data['R'][1]))
    M.t4 = Var(RangeSet(nrakes),domain=NonNegativeReals)
    M.z = Var(RangeSet(nrakes),domain=Binary)
    
    # Adding constraints
    
    M.seqa_cons = ConstraintList() # 1
    for i in range(1,nrakes+1):
        M.seqa_cons.add(expr=M.Sa[i,1]+data['t1']<=M.Sa[i,2])
        M.seqa_cons.add(expr=M.Sa[i,2]+data['t2']<=M.Sa[i,3])
        M.seqa_cons.add(expr=M.Sa[i,3]+data['t3']<=M.Sa[i,4])
    
    M.seqb_cons = ConstraintList() # 2,3,4
    for i in range(1,nrakes+1):
        M.seqb_cons.add(expr=M.Sb[i,4] >= M.Sa[i,4]+M.t4[i])
        M.seqb_cons.add(expr=M.Sb[i,3] >= M.Sb[i,4])
        M.seqb_cons.add(expr=M.Sb[i,2] >= M.Sb[i,3]+data['t3'])
        M.seqb_cons.add(expr=M.Sb[i,1] >= M.Sb[i,2]+data['t2'])
            
    M.scheda_cons = ConstraintList() # 5,6
    idx = 1
    for i in range(1,nrakes):
        for i1 in range(i+1,nrakes+1):
            for m in range(1,data['sys'][0]+1):
                M.scheda_cons.add(expr=M.Sa[i,1] >= M.Sa[i1,2]-BigM*(1-M.xa[idx,1])-BigM*(2-M.w1a[i,m]-M.w1a[i1,m]))
                M.scheda_cons.add(expr=M.Sa[i1,1] >= M.Sa[i,2]-BigM*M.xa[idx,1]-BigM*(2-M.w1a[i,m]-M.w1a[i1,m]))
            for m in range(1,data['sys'][1]+1):
                M.scheda_cons.add(expr=M.Sa[i,2] >= M.Sa[i1,3]-BigM*(1-M.xa[idx,2])-BigM*(2-M.w2a[i,m]-M.w2a[i1,m]))
                M.scheda_cons.add(expr=M.Sa[i1,2] >= M.Sa[i,3]-BigM*M.xa[idx,2]-BigM*(2-M.w2a[i,m]-M.w2a[i1,m]))
            for m in range(1,data['sys'][2]+1):
                M.scheda_cons.add(expr=M.Sa[i,3] >= M.Sa[i1,3]-BigM*(1-M.xa[idx,3])-BigM*(2-M.y[i,m]-M.y[i1,m]))
                M.scheda_cons.add(expr=M.Sa[i1,3] >= M.Sa[i,3]-BigM*M.xa[idx,3]-BigM*(2-M.y[i,m]-M.y[i1,m]))
            for m in range(1,sum(data['sys'][3])+1):
                M.scheda_cons.add(expr=M.Sa[i,4] >= M.Sb[i1,3]-BigM*(1-M.xa[idx,4])-BigM*(2-M.w4a[i,m]-M.w4a[i1,m]))
                M.scheda_cons.add(expr=M.Sa[i1,4] >= M.Sb[i,3]-BigM*M.xa[idx,4]-BigM*(2-M.w4a[i,m]-M.w4a[i1,m]))            
            idx += 1
    
    M.schedb_cons = ConstraintList() # 7,8
    idx = 1
    for i in range(1,nrakes):
        for i1 in range(i+1,nrakes+1):
            for m in range(1,data['sys'][2]+1):
                M.schedb_cons.add(expr=M.Sb[i,3] >= M.Sb[i1,2]-BigM*(1-M.xb[idx,3]))-BigM*(2-M.y[i,m]-M.y[i1,m])
                M.schedb_cons.add(expr=M.Sb[i1,3] >= M.Sb[i,2]-BigM*(M.xb[idx,3]))-BigM*(2-M.y[i,m]-M.y[i1,m])
            for m in range(1,data['sys'][1]+1):
                M.schedb_cons.add(expr=M.Sb[i,2] >= M.Sb[i1,1]-BigM*(1-M.xb[idx,2])-BigM*(2-M.w2b[i,m]-M.w2b[i1,m]))
                M.schedb_cons.add(expr=M.Sb[i1,2] >= M.Sb[i,1]-BigM*(M.xb[idx,2])-BigM*(2-M.w2b[i,m]-M.w2b[i1,m]))
            for m in range(1,data['sys'][0]+1):
                M.schedb_cons.add(expr=M.Sb[i,1] >= M.Sb[i1,1]+data['t1']-BigM*(1-M.xb[idx,1])-BigM*(2-M.w1b[i,m]-M.w1b[i1,m]))
                M.schedb_cons.add(expr=M.Sb[i1,1] >= M.Sb[i,1]+data['t1']-BigM*(M.xb[idx,1])-BigM*(2-M.w1b[i,m]-M.w1b[i1,m]))
            idx += 1              
    
    
           
    M.releases_cons = ConstraintList() # 9
    for i in range(1,nrakes+1):
        M.releases_cons.add(expr=M.Sa[i,1] >= data['a'][i-1])
    
    M.sumw1a_cons = ConstraintList() # 10.1
    for i in range(1,nrakes+1):
        M.sumw1a_cons.add(expr=sum(M.w1a[i,m] for m in range(1,data['sys'][0]+1))==1)
    
    M.sumw2a_cons = ConstraintList() # 10.2
    for i in range(1,nrakes+1):
        M.sumw2a_cons.add(expr=sum(M.w2a[i,m] for m in range(1,data['sys'][1]+1))==1)
        
    M.sumw4a_cons = ConstraintList() # 10.4
    for i in range(1,nrakes+1):
        M.sumw4a_cons.add(expr=sum(M.w4a[i,m] for m in range(1,sum(data['sys'][3])+1))==1)
    
    M.sumw1b_cons = ConstraintList() # 11.1
    for i in range(1,nrakes+1):
        M.sumw1b_cons.add(expr=sum(M.w1b[i,m] for m in range(1,data['sys'][0]+1))==1)
        
    M.sumw2b_cons = ConstraintList() # 11.2
    for i in range(1,nrakes+1):
        M.sumw2b_cons.add(expr=sum(M.w2b[i,m] for m in range(1,data['sys'][1]+1))==1)

    M.destlineassign_cons = ConstraintList() # 12
    for i in range(1,nrakes+1):
        for d in range(1,ndest+1):
            for m in range(1,data['sys'][0]+1):
                M.destlineassign_cons.add(expr=M.w1b[i,m] >= M.v[i,d]*data['l'][d-1,m-1])
                
#     M.lineproctime_cons = ConstraintList() # 13
#     for i in range(1,nrakes+1):
#         M.lineproctime_cons.add(expr=M.t4[i] == sum(sum(M.n[i,d,t] for d in range(1,ndest+1))for t in range(1,nterm+1))/data['q']+data['tau'])
    
    M.sumv_cons = ConstraintList() # 14
    for i in range(1,nrakes+1):
        M.sumv_cons.add(expr=sum(M.v[i,d] for d in range(1,ndest+1))==1)
    
    M.sumy_cons = ConstraintList() # 15
    for i in range(1,nrakes+1):
        M.sumy_cons.add(expr=sum(M.y[i,t] for t in range(1,nterm+1))==1)
        
    M.BPCdest_cons = ConstraintList() # 16
    for i in range(1,nrakes+1):
        for d in range(1,ndest+1):
            M.BPCdest_cons.add(expr=M.v[i,d] >= data['b'][i-1,d-1])
    
    M.minmaxload_cons = ConstraintList() # 17
    for i in range(1,nrakes+1):
        M.minmaxload_cons.add(expr=data['R'][0] <= sum(sum(M.n[i,d,t] for t in range(1,nterm+1))for d in range(1,ndest+1)))
        M.minmaxload_cons.add(expr=data['R'][1] >= sum(sum(M.n[i,d,t] for t in range(1,nterm+1))for d in range(1,ndest+1)))

    M.load_cons = ConstraintList() # 18
    for i in range(1,nrakes+1):
        for d in range(1,ndest+1):
            for t in range(1,nterm+1):
                M.load_cons.add(expr=M.n[i,d,t] <= M.v[i,d]*data['pendency'][d-1,t-1])
    
    M.ittcompute_cons = ConstraintList() # 19,20(corrected), extra1
    for i in range(1,nrakes+1):
        for t in range(1,nterm+1):
            for d in range(1,ndest+1):
                M.ittcompute_cons.add(expr=M.g[i,t] <= M.n[i,d,t]+BigM2*(1-M.y[i,t])+BigM2*(1-M.v[i,d]))
                M.ittcompute_cons.add(expr=M.g[i,t] >= M.n[i,d,t]-BigM2*(1-M.y[i,t])-BigM2*(1-M.v[i,d]))
            M.ittcompute_cons.add(expr=M.g[i,t] <= data['R'][1]*M.y[i,t])
            M.ittcompute_cons.add(expr=M.g[i,t] <= data['R'][1]*M.v[i,d])
    
    M.origins_cons = ConstraintList() # extra2
    for i in range(1,nrakes+1):
        k = data['origin'][i-1]
        M.origins_cons.add(expr=M.w1a[i,k]==1)
    
    M.lastlinks_cons = ConstraintList() # extra3
    idx = 1
    for m1 in range(1,data['sys'][2]):
        for m2 in range(1,data['sys'][3][m1-1]+1):
            for i in range(1,nrakes+1):
                M.lastlinks_cons.add(expr=M.w4a[i,idx]<=M.y[i,m1])
            idx += 1
    
    M.sumpendency_cons = ConstraintList() # extra 4
    for d in range(1,ndest+1):
        M.sumpendency_cons.add(expr= sum(sum(M.n[i,d,t] for t in range(1,nterm+1)) for i in range(1,nrakes+1)) <= sum(data['pendency'][d-1]))
    

    M.procTime_cons = ConstraintList() # extra, replace 13
    for i in range(1,nrakes+1):
        M.procTime_cons.add(expr=sum(M.g[i,t] for t in range(1,nterm+1)) <= data['Pure min limit']+BigM2*(1-M.z[i]))
        M.procTime_cons.add(expr=sum(M.g[i,t] for t in range(1,nterm+1)) >= data['Pure max limit']-BigM2*(M.z[i]))
        M.procTime_cons.add(expr=M.t4[i] == data['Loading times'][1]*M.z[i]+data['Loading times'][0]*(1-M.z[i]))

#     M.removethis_cons = ConstraintList()
#     for t in range(1,nterm+1):
#         M.removethis_cons.add(expr=sum(M.g[i,t] for i in range(1,nrakes+1))>=1)
    
    #M.obj = Objective(expr=data['c1']*(sum(M.Sb[i,1]-data['a'][i-1] for i in range(1,nrakes+1)))+data['c2']*(sum(sum(sum(M.n[i,d,t] for t in range(1,nterm+1))for d in range(1,ndest+1))for i in range(1,nrakes+1))-sum(sum(M.g[i,t] for i in range(1,nrakes+1))for t in range(1,nterm+1)))-data['c3']*(sum(sum(sum(M.n[i,d,t] for t in range(1,nterm+1))for d in range(1,ndest+1))for i in range(1,nrakes+1))),sense=minimize)
    #M.obj = Objective(expr=data['c1']*(sum(M.Sb[i,1]-data['a'][i-1] for i in range(1,nrakes+1)))-data['c2']*(sum(sum(M.g[i,t] for i in range(1,nrakes+1)) for t in range(1,nterm+1))),sense=minimize)
    M.obj = Objective(expr=data['c1']*(sum(M.Sb[i,1]-data['a'][i-1] for i in range(1,nrakes+1)))+data['cont cost']*sum(sum(sum(M.n[i,d,t] for i in range(1,nrakes+1)) for d in range(1,ndest+1))for t in range(1,nterm+1)),sense=maximize)
    
    return M


In [3]:
data = {'pendency':[],
        'b':[],
        'q':20.0,
        'sys':[2,3,2,[3,3]],
        'a':[],
        'l':[],
        'origin':[],
        'R':[80,90],
        'Pure min limit': 89,
        'Pure max limit': 90,
        'Loading times': [0.5,20.0],
        'cont cost': 1.0,
        'tau':1.5,
        't1':2.0,
        't2':2.0,
        't3':2.0,
        'c1':-150.0,
        'c2':1.0,
        'c3':75.0,
        'BigM':48.0,
        'BigM2':180.0}

nrakes = 10
ndest = 6
nterm = 2
l = np.array([[1,0],[1,0],[1,0],[0,1],[0,1],[0,1]])
arr_times = []
s = 0
for i in range(nrakes):
    arr_times.append(s)
    s += np.random.exponential(2.0)
bpc = np.array([[0.0 for d in range(ndest)]for i in range(nrakes)])
bpc[4,4] = 1.0
origin = [2, 1, 2, 2, 1, 2, 1, 1, 1, 2]
data['b'] = bpc
data['origin'] = origin
data['a'] = arr_times
data['l'] = l

p = pd.read_csv('data/d.csv',header=None).to_numpy()
data['pendency'] = p
print(p)

[[100 140]
 [  0 200]
 [178  32]
 [ 63 113]
 [ 80 150]
 [ 50 162]]


In [4]:
model1 = create_model(data)

opt = SolverFactory('cplex')
result = opt.solve(model1,tee=True)
print("Solver status :",result.solver.status)
print("Termination condition :",result.solver.termination_condition)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 20.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2020.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\91824\AppData\Local\Temp\tmp_aksigld.cplex.log' open.
CPLEX> Problem 'C:\Users\91824\AppData\Local\Temp\tmppwzwd207.pyomo.lp' read.
Read time = 0.00 sec. (0.28 ticks)
CPLEX> Problem name         : C:\Users\91824\AppData\Local\Temp\tmppwzwd207.pyomo.lp
Objective sense      : Maximize
Variables            :     796  [Nneg: 91,  Binary: 565,  General Integer: 140]
Objective nonzeros   :     131
Linear constraints   :    2627  [Less: 2396,  Greater: 140,  Equal: 91]
  Nonzeros           :   11061
  RHS nonzeros       :    2147

Variables            : Min LB: 0.000000         Max UB: 90.00

 462863 15017   -38770.0458    15   -39016.4706   -38578.4877  1737291    1.12%
 474138 16306   -38708.8082     8   -39016.4706   -38596.3004  1818522    1.08%
 488313 18779   -38663.1736     8   -39016.4706   -38642.4280  1881289    0.96%
 504615 21025   -38663.1736     3   -39016.4706   -38663.1736  1930604    0.91%
 517853 23782   -38663.1736     4   -39016.4706   -38663.1736  1982178    0.91%
 536243 26583    infeasible         -39016.4706   -38663.1736  2038189    0.91%
 549564 28792   -38663.1736     3   -39016.4706   -38663.1736  2086950    0.91%
 568011 30960   -38693.1736     3   -39016.4706   -38663.1736  2129474    0.91%
 581787 33246   -38693.1736     1   -39016.4706   -38663.1736  2174904    0.91%
 598821 36414    infeasible         -39016.4706   -38663.1736  2232689    0.91%
Elapsed time = 79.42 sec. (42047.66 ticks, tree = 32.57 MB, solutions = 9)
 613071 38843   -38663.1736     6   -39016.4706   -38663.1736  2286899    0.91%
 629130 42399   -39008.4852     4   -39016.47

In [5]:
result_rows = []
for i in range(1,nrakes+1):
    res1 = [data['a'][i-1]]+[max(0,model1.Sa[i,j].value) for j in range(1,5)]+[model1.Sb[i,4-j].value for j in range(0,4)]+[model1.t4[i].value]
    allocations = []
    l = []
    for m in range(1,data['sys'][0]+1):
        if model1.w1a[i,m].value > 0.5:
            l.append(m)
    if len(l) > 1:
        print("Error 1",l)
    else:
        allocations.append(l[0])
    l = []
    for m in range(1,data['sys'][1]+1):
        if model1.w2a[i,m].value > 0.5:
            l.append(m)
    if len(l) > 1:
        print("Error 2",l)
    else:
        allocations.append(l[0])
    l = []
    for m in range(1,sum(data['sys'][3])+1):
        if model1.w4a[i,m].value > 0.5:
            l.append(m)
    if len(l) > 1:
        print("Error 2",l)
    else:
        allocations.append(l[0])
    l = []
    for m in range(1,data['sys'][1]+1):
        if model1.w2b[i,m].value > 0.5:
            l.append(m)
    if len(l) > 1:
        print("Error 2",l)
    else:
        allocations.append(l[0])
    l = []
    for m in range(1,data['sys'][0]+1):
        if model1.w1b[i,m].value > 0.5:
            l.append(m)
    if len(l) > 1:
        print("Error 1",l)
    else:
        allocations.append(l[0])
    l = []
    for d in range(1,ndest+1):
        if model1.v[i,d].value > 0.5:
            l.append(d)
    if len(l) > 1:
        print("Error 1",l)
    else:
        allocations.append(l[0])

    res1 += allocations

    res1.append(sum([model1.n[i,d,1].value for d in range(1,ndest+1)]))
    res1.append(sum([model1.n[i,d,2].value for d in range(1,ndest+1)]))
    l = []
    for m in range(1,data['sys'][2]+1):
        if model1.y[i,m].value > 0.5:
            l.append(m)
    if len(l) > 1:
        print("Error 2",l)
    else:
        res1.append(l[0])
    for t in range(1,nterm+1):
        res1.append(model1.g[i,t].value)
    result_rows.append(res1)

result_dataframe = pd.DataFrame(data=result_rows,columns=['Arr times','Sa 1','Sa 2','Sa 3','Sa 4','Sb 4','Sb 3','Sb 2','Sb 1','t4','alloca 1','alloca 2','alloca 4','allocb 2','allocb 1','dest','pend 1','pend 2','terminal allocated']+['g'+str(t) for t in range(1,nterm+1)])
result_dataframe.to_csv('data/dummy_result.csv')
result_dataframe.head(10)

,Arr times,Sa 1,Sa 2,Sa 3,Sa 4,Sb 4,Sb 3,Sb 2,Sb 1,t4,...,alloca 2,alloca 4,allocb 2,allocb 1,dest,pend 1,pend 2,terminal allocated,g1,g2
0,0.000000,0.000000,2.000000,4.000000,6.000000,26.000000,26.000000,28.000000,30.000000,20.0,...,1,1,1,2,5,0.000000e+00,90.0,1,0.0,0.0
1,1.591749,2.000000,4.000000,6.000000,8.000000,28.000000,28.000000,30.000000,32.000000,20.0,...,2,3,1,1,1,-3.614774e-08,90.0,1,0.0,0.0
2,4.103611,4.103611,6.103611,8.103611,10.103611,30.103611,30.103611,32.103611,34.103611,20.0,...,1,4,1,1,2,0.000000e+00,90.0,1,0.0,0.0
3,5.950057,6.103611,8.103611,10.103611,12.103611,12.603611,12.603611,14.603611,16.603611,0.5,...,1,6,1,2,6,0.000000e+00,90.0,2,0.0,90.0
4,7.524459,7.524459,9.524459,11.524459,13.524459,33.524459,33.524459,35.524459,37.524459,20.0,...,2,2,1,2,5,0.000000e+00,90.0,1,0.0,0.0
5,8.626556,8.626556,10.626556,12.626556,14.626556,15.126556,15.126556,17.126556,19.126556,0.5,...,1,5,1,2,6,0.000000e+00,90.0,2,0.0,90.0
6,10.702875,10.702875,12.702875,14.702875,16.702875,36.702875,36.702875,38.702875,40.702875,20.0,...,1,6,1,1,1,9.000000e+01,0.0,2,0.0,0.0
7,11.853913,12.702875,14.702875,16.702875,18.702875,38.702875,38.702875,40.702875,42.702875,20.0,...,1,5,1,1,3,9.000000e+01,0.0,2,0.0,0.0
8,16.300962,16.300962,18.300962,20.300962,26.000000,46.000000,46.000000,48.000000,50.000000,20.0,...,1,1,1,2,4,0.000000e+00,90.0,1,0.0,0.0
9,22.930786,22.930786,24.930786,26.930786,28.930786,48.930786,48.930786,50.930786,52.930786,20.0,...,1,3,1,1,2,0.000000e+00,90.0,1,0.0,0.0


In [8]:
model1.pprint()

34 Set Declarations
    BPCdest_cons_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   60 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60}
    Sa_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain                : Size : Members
        None :     2 : Sa_index_0*Sa_index_1 :   40 : {(1, 1), (1, 2), (1, 3), (1, 4), (2, 1), (2, 2), (2, 3), (2, 4), (3, 1), (3, 2), (3, 3), (3, 4), (4, 1), (4, 2), (4, 3), (4, 4), (5, 1), (5, 2), (5, 3), (5, 4), (6, 1), (6, 2), (6, 3), (6, 4), (7, 1), (7, 2), (7, 3), (7, 4), (8, 1), (8, 2), (8, 3), (8, 4), (9, 1), (9, 2), (9, 3), (9, 4), (10, 1), (10, 2), (10, 3), (10, 4)}
    Sb_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain                : Size : Members


        Key : Lower : Body                                                              : Upper : Active
          1 :  -Inf :     g[1,1] - (n[1,1,1] + 180.0*(1 - y[1,1]) + 180.0*(1 - v[1,1])) :   0.0 :   True
          2 :  -Inf :       n[1,1,1] - 180.0*(1 - y[1,1]) - 180.0*(1 - v[1,1]) - g[1,1] :   0.0 :   True
          3 :  -Inf :     g[1,1] - (n[1,2,1] + 180.0*(1 - y[1,1]) + 180.0*(1 - v[1,2])) :   0.0 :   True
          4 :  -Inf :       n[1,2,1] - 180.0*(1 - y[1,1]) - 180.0*(1 - v[1,2]) - g[1,1] :   0.0 :   True
          5 :  -Inf :     g[1,1] - (n[1,3,1] + 180.0*(1 - y[1,1]) + 180.0*(1 - v[1,3])) :   0.0 :   True
          6 :  -Inf :       n[1,3,1] - 180.0*(1 - y[1,1]) - 180.0*(1 - v[1,3]) - g[1,1] :   0.0 :   True
          7 :  -Inf :     g[1,1] - (n[1,4,1] + 180.0*(1 - y[1,1]) + 180.0*(1 - v[1,4])) :   0.0 :   True
          8 :  -Inf :       n[1,4,1] - 180.0*(1 - y[1,1]) - 180.0*(1 - v[1,4]) - g[1,1] :   0.0 :   True
          9 :  -Inf :     g[1,1] - (n[1,5,1] + 180.0*(1

         16 :  -Inf :   n[2,2,2] - 200*v[2,2] :   0.0 :   True
         17 :  -Inf :   n[2,3,1] - 178*v[2,3] :   0.0 :   True
         18 :  -Inf :    n[2,3,2] - 32*v[2,3] :   0.0 :   True
         19 :  -Inf :    n[2,4,1] - 63*v[2,4] :   0.0 :   True
         20 :  -Inf :   n[2,4,2] - 113*v[2,4] :   0.0 :   True
         21 :  -Inf :    n[2,5,1] - 80*v[2,5] :   0.0 :   True
         22 :  -Inf :   n[2,5,2] - 150*v[2,5] :   0.0 :   True
         23 :  -Inf :    n[2,6,1] - 50*v[2,6] :   0.0 :   True
         24 :  -Inf :   n[2,6,2] - 162*v[2,6] :   0.0 :   True
         25 :  -Inf :   n[3,1,1] - 100*v[3,1] :   0.0 :   True
         26 :  -Inf :   n[3,1,2] - 140*v[3,1] :   0.0 :   True
         27 :  -Inf :                n[3,2,1] :   0.0 :   True
         28 :  -Inf :   n[3,2,2] - 200*v[3,2] :   0.0 :   True
         29 :  -Inf :   n[3,3,1] - 178*v[3,3] :   0.0 :   True
         30 :  -Inf :    n[3,3,2] - 32*v[3,3] :   0.0 :   True
         31 :  -Inf :    n[3,4,1] - 63*v[3,4] :   0.0 :

        Key  : Lower : Body                                                                       : Upper : Active
           1 :  -Inf :    Sa[2,2] - 48.0*(1 - xa[1,1]) - 48.0*(2 - w1a[1,1] - w1a[2,1]) - Sa[1,1] :   0.0 :   True
           2 :  -Inf :          Sa[1,2] - 48.0*xa[1,1] - 48.0*(2 - w1a[1,1] - w1a[2,1]) - Sa[2,1] :   0.0 :   True
           3 :  -Inf :    Sa[2,2] - 48.0*(1 - xa[1,1]) - 48.0*(2 - w1a[1,2] - w1a[2,2]) - Sa[1,1] :   0.0 :   True
           4 :  -Inf :          Sa[1,2] - 48.0*xa[1,1] - 48.0*(2 - w1a[1,2] - w1a[2,2]) - Sa[2,1] :   0.0 :   True
           5 :  -Inf :    Sa[2,3] - 48.0*(1 - xa[1,2]) - 48.0*(2 - w2a[1,1] - w2a[2,1]) - Sa[1,2] :   0.0 :   True
           6 :  -Inf :          Sa[1,3] - 48.0*xa[1,2] - 48.0*(2 - w2a[1,1] - w2a[2,1]) - Sa[2,2] :   0.0 :   True
           7 :  -Inf :    Sa[2,3] - 48.0*(1 - xa[1,2]) - 48.0*(2 - w2a[1,2] - w2a[2,2]) - Sa[1,2] :   0.0 :   True
           8 :  -Inf :          Sa[1,3] - 48.0*xa[1,2] - 48.0*(2 - w2a[1,2] - w2

        Key : Lower : Body                                                                             : Upper : Active
          1 :  -Inf :                                           Sb[2,2] - 48.0*(1 - xb[1,3]) - Sb[1,3] :   0.0 :   True
          2 :  -Inf :                                                 Sb[1,2] - 48.0*xb[1,3] - Sb[2,3] :   0.0 :   True
          3 :  -Inf :                                           Sb[2,2] - 48.0*(1 - xb[1,3]) - Sb[1,3] :   0.0 :   True
          4 :  -Inf :                                                 Sb[1,2] - 48.0*xb[1,3] - Sb[2,3] :   0.0 :   True
          5 :  -Inf :          Sb[2,1] - 48.0*(1 - xb[1,2]) - 48.0*(2 - w2b[1,1] - w2b[2,1]) - Sb[1,2] :   0.0 :   True
          6 :  -Inf :                Sb[1,1] - 48.0*xb[1,2] - 48.0*(2 - w2b[1,1] - w2b[2,1]) - Sb[2,2] :   0.0 :   True
          7 :  -Inf :          Sb[2,1] - 48.0*(1 - xb[1,2]) - 48.0*(2 - w2b[1,2] - w2b[2,2]) - Sb[1,2] :   0.0 :   True
          8 :  -Inf :                Sb[

In [17]:
def get_specTrains(result):
    # need to get rows where atleast one of the elelments is greater than 24
    r = result_dataframe.loc[:,result_dataframe.columns[:9]]
    ans = []
    for i in r.index:
        for j in r.columns:
            if r.loc[i,j] >= 24.0:
                ans.append(r.loc[i,:])
                break
    ans = pd.DataFrame(data=ans,columns=r.columns)
    return ans